## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
# Import API key
from config import g_key
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,84.20,84,75,9.22,broken clouds
1,1,Butaritari,KI,3.0707,172.7902,81.23,79,71,20.20,broken clouds
2,2,Klaeng,TH,12.7797,101.6483,75.99,89,2,2.35,clear sky
3,3,Narsaq,GL,60.9167,-46.0500,23.16,81,100,3.65,overcast clouds
4,4,Aklavik,CA,68.2191,-135.0107,-7.60,71,90,11.50,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)


,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,84.20,84,75,9.22,broken clouds
1,1,Butaritari,KI,3.0707,172.7902,81.23,79,71,20.20,broken clouds
2,2,Klaeng,TH,12.7797,101.6483,75.99,89,2,2.35,clear sky
7,7,Bogo,PH,11.0517,124.0055,81.73,77,99,12.55,overcast clouds
10,10,Rikitea,PF,-23.1203,-134.9692,78.78,68,45,9.57,scattered clouds
11,11,Hithadhoo,MV,-0.6000,73.0833,81.36,75,41,13.96,scattered clouds
13,13,Samarai,PG,-10.6167,150.6667,86.50,67,63,4.45,broken clouds
16,16,Souillac,MU,-20.5167,57.5167,77.00,83,40,4.61,scattered clouds
20,20,Mataura,NZ,-46.1927,168.8643,80.42,46,0,1.88,clear sky
24,24,Kilindoni,TZ,-7.9139,39.6668,81.36,79,90,11.32,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.

# 4a. Determine if there are any empty rows.
# clean_df.isnull().sum()
clean_df.count()


Unnamed: 0             190
City                   190
Country                189
Lat                    190
Lng                    190
Max Temp               190
Humidity               190
Cloudiness             190
Wind Speed             190
Current Description    190
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# clean_df2 = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
#                                        (city_data_df["Max Temp"] >= min_temp)].dropna()
clean_df2 = clean_df.dropna()
clean_df2.count()
# clean_df2.isnull().sum()


Unnamed: 0             189
City                   189
Country                189
Lat                    189
Lng                    189
Max Temp               189
Humidity               189
Cloudiness             189
Wind Speed             189
Current Description    189
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,84.20,broken clouds,-21.2000,-175.2000,
1,Butaritari,KI,81.23,broken clouds,3.0707,172.7902,
2,Klaeng,TH,75.99,clear sky,12.7797,101.6483,
7,Bogo,PH,81.73,overcast clouds,11.0517,124.0055,
10,Rikitea,PF,78.78,scattered clouds,-23.1203,-134.9692,
11,Hithadhoo,MV,81.36,scattered clouds,-0.6000,73.0833,
13,Samarai,PG,86.50,broken clouds,-10.6167,150.6667,
16,Souillac,MU,77.00,scattered clouds,-20.5167,57.5167,
20,Mataura,NZ,80.42,clear sky,-46.1927,168.8643,
24,Kilindoni,TZ,81.36,overcast clouds,-7.9139,39.6668,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):

        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:


hotel_df.tail(20)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
625,Payo,ID,75.45,light rain,-3.7508,103.6372,Mess keban agung
629,Ifo,NG,79.77,broken clouds,6.8144,3.1958,Peace Land Hotel
640,Ugoofaaru,MV,81.39,few clouds,5.6667,73.0000,Ungoofaaru Inn
647,Mudgee,AU,75.00,broken clouds,-32.5943,149.5871,Parklands Resort & Conference Centre
648,Tagusao,PH,80.96,overcast clouds,9.1924,117.8141,Toledo
651,Kingaroy,AU,86.00,overcast clouds,-26.5333,151.8333,Ascot Lodge Motel Kingaroy
655,Mayor Pablo Lagerenza,PY,75.15,broken clouds,-19.9309,-60.7718,
656,Ballina,AU,84.20,overcast clouds,-28.8667,153.5667,Ramada Hotel & Suites by Wyndham Ballina Byron
657,Dhidhdhoo,MV,81.30,clear sky,6.8833,73.1000,Dhidhoo beach
663,Male,MV,80.60,few clouds,4.1748,73.5089,Hotel Octave Maldives


In [10]:
# 7. Drop the rows where there is no Hotel Name.

new_df = hotel_df[hotel_df['Hotel Name']!= ""]
new_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,84.20,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
1,Butaritari,KI,81.23,broken clouds,3.0707,172.7902,Isles Sunset Lodge
2,Klaeng,TH,75.99,clear sky,12.7797,101.6483,Damrongcheep in Resort
7,Bogo,PH,81.73,overcast clouds,11.0517,124.0055,Acrige Apartelle 2 pax Twin @ heart of Bogo City
10,Rikitea,PF,78.78,scattered clouds,-23.1203,-134.9692,Pension Maro'i
...,...,...,...,...,...,...,...
689,Abangay,PH,80.65,scattered clouds,10.9667,122.6500,OYO 236 Hotel Edmundo
691,Gorontalo,ID,82.90,overcast clouds,0.5412,123.0595,Maqna Hotel by Prasanthi
692,Neiafu,TO,78.80,moderate rain,-18.6500,-173.9833,Mystic Sands
694,Jocoro,SV,76.21,clear sky,13.6167,-88.0167,Bambiu_22


In [11]:
# 8a. Create the output File (CSV)

output_data_file = "Weather_Database/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))